In [2]:
from tqdm.notebook import tqdm
import pickle, gzip
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import torch

import nibabel as nib
from nilearn import maskers
from nilearn.plotting import view_img
from nilearn.image import resample_img

from neurovlm.data import get_data_dir
from neurovlm.models import Specter
from neurovlm.llm_summary import generate_response, search_papers, search_wiki
from neurovlm.brain2text import run_brain2text

In [3]:
get_data_dir()

PosixPath('/Users/borng/code/lab_work/neurovlm/src/neurovlm/neurovlm_data')

In [4]:
# Load network atlases
with gzip.open(get_data_dir() / f"networks_arrays.pkl.gz", "rb") as f:
    networks = pickle.load(f)

In [5]:
# Load models
proj_head_mse_adhoc = torch.load(get_data_dir() / f"proj_head_image_infonce.pt", weights_only=False).cpu()
proj_head_img = torch.load(get_data_dir() / f"proj_head_image_infonce.pt", weights_only=False).cpu()
proj_head_text = torch.load(get_data_dir() / f"proj_head_text_infonce.pt", weights_only=False).cpu()
specter = Specter("allenai/specter2_aug2023refresh", adapter="adhoc_query")
autoencoder = torch.load(get_data_dir() / "autoencoder_sparse.pt", weights_only=False).cpu()
# decoder = autoencoder.decoder.to("cpu")
mask_arrays = np.load(f"{get_data_dir()}/mask.npz", allow_pickle=True)
mask_img = nib.Nifti1Image(mask_arrays["mask"].astype(float),  mask_arrays["affine"])
masker = maskers.NiftiMasker(mask_img=mask_img, dtype=np.float32).fit()

There are adapters available but none are activated for the forward pass.


In [6]:
networks = {k: v for _k in networks.keys() for k, v in networks[_k].items()}

In [7]:
mask_arrays = np.load(f"{get_data_dir()}/mask.npz", allow_pickle=True)
mask_img = nib.Nifti1Image(mask_arrays["mask"].astype(float),  mask_arrays["affine"])
masker = maskers.NiftiMasker(mask_img=mask_img, dtype=np.float32).fit()
networks_resampled = {}

for k in tqdm(networks.keys(), total=len(networks)):
    img = nib.Nifti1Image(networks[k]["array"], affine=networks[k]["affine"])

    if len(np.unique(networks[k]["array"])) == 2:
        # binary data
        img_resampled = resample_img(img, mask_arrays["affine"], interpolation="nearest")
    else:
        img_resampled = resample_img(img, mask_arrays["affine"])
        img_resampled_arr = img_resampled.get_fdata()
        img_resampled_arr[img_resampled_arr < 0] = 0.
        thresh = np.percentile(img_resampled_arr.flatten(), 95)
        img_resampled_arr[img_resampled_arr < thresh] = 0.
        img_resampled_arr[img_resampled_arr >= thresh] = 1.
        img_resampled = nib.Nifti1Image(img_resampled_arr, affine=mask_arrays["affine"])

    networks_resampled[k] = img_resampled

  0%|          | 0/152 [00:00<?, ?it/s]

In [8]:
networks_embed = {}

for k, v in tqdm(networks_resampled.items(), total=len(networks_resampled)):
    networks_embed[k] = autoencoder.encoder(torch.from_numpy(masker.transform(v)))

  0%|          | 0/152 [00:00<?, ?it/s]

## Brain to text (Network vs Paper)

In [9]:
df_similar_titles = pd.DataFrame(columns=[
    "atlas_label",
    "similar_title1",
    "similar_title2",
    "similar_title3",
    "similar_title4",
    "similar_title5"
])

df_similar_titles.head()

def add2df(df, atlas_label, similar_titles):
    new_row = {
        "atlas_label": atlas_label,
        "similar_title1": similar_titles[0] if len(similar_titles) > 0 else np.nan,
        "similar_title2": similar_titles[1] if len(similar_titles) > 1 else np.nan,
        "similar_title3": similar_titles[2] if len(similar_titles) > 2 else np.nan,
        "similar_title4": similar_titles[3] if len(similar_titles) > 3 else np.nan,
        "similar_title5": similar_titles[4] if len(similar_titles) > 4 else np.nan,
    }
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    return df

In [ ]:
# i need non aligned tan tent text embeddings
# the wiki aligned ones are not working well
# i also dont have the most recent publication parquet file
abstract, titles = search_papers(networks_embed["AUD"], show_titles=True)

Top matches:
1. Remembering with gains and losses: effects of monetary reward and punishment on successful encoding activation of source memories.
2. Amygdala response to facial expressions reflects emotional learning.
3. The functional and structural neural basis of individual differences in loss aversion.
4. Two Sides of the Same Coin: Learning via Positive and Negative Reinforcers in the Human Striatum
5. Threat of punishment motivates memory encoding via amygdala, not midbrain, interactions with the medial temporal lobe.


In [18]:
for key, item in tqdm(networks_embed.items(), total=len(networks_embed)):
    abstract, titles = search_papers(item)
    df_similar_titles = add2df(df_similar_titles, key, titles)

  0%|          | 0/152 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (27501x384 and 1x384)

In [19]:
df_similar_titles

,atlas_label,similar_title1,similar_title2,similar_title3,similar_title4,similar_title5
0,VIS-P,Altered synchronous neural activities in retin...,Abnormal regional spontaneous neural activity ...,Altered spontaneous brain activity pattern in ...,Study of vision‐related resting‐state activity...,Differential Responses to a Visual Self-Motion...
1,CG-OP,Common cortical network for first and second p...,A simultaneous EEG-fMRI study of painful elect...,Ongoing first-hand pain facilitates somatosens...,Mechanical noxious stimuli cause bilateral act...,Placebo Effect on Modulating Empathic Pain: Re...
2,DN-B,Intrinsic Default Mode Network Connectivity Pr...,"Emotional, especially negative microblogs are ...",Nodes of the default mode network implicated i...,Differential patterns of default mode network ...,Patterns of brain activity supporting autobiog...
3,SMOT-B,Cortical representation of experimental tooth ...,The Association Between Genetic Variation in F...,Loss of inhibition in ipsilateral somatosensor...,Cortical Thickness Alterations in Patients Wit...,Steady-state activation in somatosensory corte...
4,AUD,Processing the acoustic effect of size in spee...,Cortical dynamics of acoustic and phonological...,Preference for Audiovisual Speech Congruency i...,Hierarchical organization of human auditory co...,Hemispheric specialization for processing audi...
5,PM-PPr,Cortical processing of lateral skin stretch st...,Processing in prefrontal cortex underlies tact...,Simultaneous electroencephalography and functi...,Bilateral parietal cortex function during moto...,Modulation of somatosensory processing by action.
6,dATN-B,Individual face- and house-related eye movemen...,Investigating the Impact of the Missing Signif...,fMR-adaptation reveals separate processing reg...,Functional specialization and generalization f...,Extrastriate Visual Areas Integrate Form Featu...
7,SMOT-A,Source Analysis of Beta-Synchronisation and Co...,Post-movement beta rebound is generated in mot...,Effect of repetitive transcranial magnetic sti...,Non-invasive brain mapping of motor-related ar...,Resting-State Fluctuations of EEG Sensorimotor...
8,LANG,Intonation guides sentence processing in the l...,Neural correlates of second-language communica...,Neural bases of social communicative intention...,Neural substrates of sarcasm: a functional mag...,Advantages of sentence-level fMRI language tas...
9,FPN-B,"Emotional, especially negative microblogs are ...",Functional Realignment of Frontoparietal Subne...,Competition between frontoparietal control and...,Disruptions of frontoparietal control network ...,Mentalizing in an economic games context is as...


In [20]:
df_similar_titles.to_csv("~/Desktop/brain2text_results.csv", index=False)